In [26]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import tensorflow as tf
import os

# import image classification libraries
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [42]:
batch_size = 32
image_size = 512
num_classes = 5
epochs = 5
auto = tf.data.AUTOTUNE

In [46]:
re = tf.random.Generator.from_seed(42, alg='philox')

In [ ]:
# import compile with generator

## Importing data

In [28]:
train = 'data\\tfrecords-jpeg-512x512\\train'

In [29]:
test = 'data\\tfrecords-jpeg-224x224\\test'

In [30]:
val = 'data\\tfrecords-jpeg-512x512\\val'

In [31]:
# classes
CLASSES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips', 'waterlilies']